In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [2]:
df = pd.read_csv('data.csv')
df.sample(5)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
5093,Infiniti,G Coupe,2012,premium unleaded (recommended),330.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Midsize,Coupe,25,17,190,44200
2929,Toyota,Corolla,2015,regular unleaded,132.0,4.0,MANUAL,front wheel drive,4.0,NaN,Compact,Sedan,37,28,2031,16950
9003,GMC,Savana Cargo,2014,flex-fuel (unleaded/E85),285.0,8.0,AUTOMATIC,rear wheel drive,3.0,Flex Fuel,Midsize,Cargo Van,17,11,549,29215
2771,Chevrolet,Colorado,2012,regular unleaded,300.0,8.0,AUTOMATIC,rear wheel drive,4.0,Performance,Compact,Extended Cab Pickup,20,14,1385,27010
8902,Volvo,S60,2016,regular unleaded,240.0,4.0,AUTOMATIC,front wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,38,26,870,36800


In [3]:
df.columns = [i.lower().replace(' ', '_') for i in df.columns]

In [4]:
columns_to_stay = ['make', 'model', 'year', 'engine_hp',
                   'engine_cylinders', 'transmission_type',
                   'vehicle_style', 'highway_mpg', 'city_mpg', 'msrp']
df.drop(df.columns[~df.columns.isin(columns_to_stay)], axis=1, inplace=True)
df.rename({'msrp': 'price'}, axis=1, inplace=True)
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


In [5]:
df.isna().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
price                 0
dtype: int64

In [6]:
df.fillna(0, inplace=True)

##### Question 1
What is the most frequent observation (mode) for the column transmission_type?

In [7]:
df.transmission_type.value_counts()

AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: transmission_type, dtype: int64

##### Question 2
What are the two features that have the biggest correlation in this dataset?

In [8]:
round(df.corr(),2)

C:\Users\49162\AppData\Local\Temp\ipykernel_21196\2139689135.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  round(df.corr(),2)


,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.00,0.34,-0.04,0.26,0.20,0.23
engine_hp,0.34,1.00,0.77,-0.42,-0.42,0.65
engine_cylinders,-0.04,0.77,1.00,-0.61,-0.59,0.53
highway_mpg,0.26,-0.42,-0.61,1.00,0.89,-0.16
city_mpg,0.20,-0.42,-0.59,0.89,1.00,-0.16
price,0.23,0.65,0.53,-0.16,-0.16,1.00


Make price binary (above & below average)

In [9]:
df['above_average'] = (df.price > df.price.mean()).astype(int)
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135,1
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650,1
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350,0
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450,0
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500,0


Split the data <br>
Split your data in train/val/test sets with 60%/20%/20% distribution.<br>
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42. <br>

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
price = df['price'].copy()
df.drop('price', axis=1, inplace=True)

In [12]:
df_train_val, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_val, test_size=0.25, random_state=42)
len(df_train), len(df_val), len(df_test)

(7148, 2383, 2383)

In [13]:
def x_y_split_and_index(df, y_column):
    df.reset_index(drop=True, inplace=True)
    cols_x = df.columns[df.columns!=y_column]
    X = df[cols_x].copy()
    y = df[y_column].values
    return X, y

In [14]:
del df_train_val
X_train, y_train = x_y_split_and_index(df_train, 'above_average')
X_test, y_test = x_y_split_and_index(df_test, 'above_average')
X_val, y_val = x_y_split_and_index(df_val, 'above_average')

In [15]:
X_train

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg
0,Mitsubishi,Endeavor,2011,225.0,6.0,AUTOMATIC,4dr SUV,19,15
1,Kia,Borrego,2009,276.0,6.0,AUTOMATIC,4dr SUV,21,17
2,Lamborghini,Gallardo,2012,570.0,10.0,MANUAL,Convertible,20,12
3,Chevrolet,Colorado,2016,200.0,4.0,AUTOMATIC,Crew Cab Pickup,27,20
4,Pontiac,Vibe,2009,158.0,4.0,AUTOMATIC,4dr Hatchback,26,20
...,...,...,...,...,...,...,...,...,...
7143,Toyota,Sienna,2016,266.0,6.0,AUTOMATIC,Passenger Minivan,25,18
7144,Chevrolet,HHR,2009,260.0,4.0,MANUAL,Wagon,29,21
7145,Hyundai,Veracruz,2012,260.0,6.0,AUTOMATIC,4dr SUV,22,17
7146,Mitsubishi,Expo,1993,136.0,4.0,MANUAL,2dr Hatchback,26,19


##### Question 3

Which of these variables has the lowest mutual information score (between above_average and other categorical variables)?

In [16]:
from sklearn.metrics import mutual_info_score

In [17]:
categorical = ['make', 'model', 'transmission_type', 'vehicle_style']
for i in categorical:
    print(i, round(mutual_info_score(X_train[i], y_train), 2))

make 0.24
model 0.46
transmission_type 0.02
vehicle_style 0.08


##### Question 4
What accuracy did you get?

In [18]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [19]:
train_dicts = X_train.to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(X_val.to_dict(orient='records'))

In [20]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
first_score = accuracy_score(y_val, y_pred)
round(first_score, 2)

0.95

##### Question 5
Let's find the least useful feature using the feature elimination technique. <br>
Train a model with all these features (using the same parameters as in Q4).<br>
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.<br>
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.<br>
Which of following feature has the smallest difference?

In [21]:
def full_pipeline(model_obj, vectorizer_obj, score_obj, df, y_column):
    # split df
    df_train_val, df_test = train_test_split(df, test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_train_val, test_size=0.25, random_state=42)
    X_train, y_train = x_y_split_and_index(df_train, y_column)
    X_test, y_test = x_y_split_and_index(df_test, y_column)
    X_val, y_val = x_y_split_and_index(df_val, y_column)
    # OHE
    train_dicts = X_train.to_dict(orient='records')
    X_train = vectorizer_obj.fit_transform(train_dicts)
    X_val = vectorizer_obj.transform(X_val.to_dict(orient='records'))
    # model fit
    model_obj.fit(X_train, y_train)
    # prediction
    y_pred = model_obj.predict(X_val)
    # scoring
    sc_ = score_obj(y_val, y_pred)
    return sc_

In [22]:
final_scores = {}
for i in ['year', 'engine_hp', 'transmission_type', 'city_mpg']:
    df_excl = df.drop(i, axis=1).copy()
    score = full_pipeline(model, dv, accuracy_score, df_excl, 'above_average')
    final_scores[i] = score

In [23]:
for i, j in final_scores.items():
    print(i, j - first_score)

year 0.002937473772555599
engine_hp -0.01720520352496857
transmission_type 0.0
city_mpg -0.012589173310952662


##### Question 6
For this question, we'll see how to use a linear regression model from Scikit-Learn.<br>
We'll need to use the original column price. Apply the logarithmic transformation to this column.<br>
Fit the Ridge regression model on the training data with a solver 'sag'. Set the seed to 42.<br>
This model also has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10].<br>
Round your RMSE scores to 3 decimal digits.<br>
Which of these alphas leads to the best RMSE on the validation set?



In [24]:
df = df.merge(price, left_index=True, right_index=True).copy()
df.drop('above_average', axis=1, inplace=True)
df.sample(5)

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
5375,Volkswagen,Golf GTI,2015,210.0,4.0,AUTOMATED_MANUAL,4dr Hatchback,33,25,29485
10890,Acura,TSX,2014,201.0,4.0,AUTOMATIC,Sedan,31,22,31635
10984,Toyota,Tundra,2016,381.0,8.0,AUTOMATIC,Crew Cab Pickup,18,13,40535
2257,Toyota,Camry,2017,178.0,4.0,AUTOMATIC,Sedan,33,24,23070
5133,Infiniti,G35,2006,280.0,6.0,AUTOMATIC,Sedan,23,17,31450


In [25]:
df['price'] = np.log(df.price)

In [26]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [27]:
ridge_results = {}
for a in [0, 0.01, 0.1, 1, 10]:
    model_r = Ridge(solver='sag', random_state=42, alpha=a)
    score_ridge = full_pipeline(model_r, dv, mean_squared_error, df, 'price')
    ridge_results[a] = score_ridge

ridge_results

{0: 0.24411105304796615,
 0.01: 0.24411120517873716,
 0.1: 0.24411257431694833,
 1: 0.24412626251194341,
 10: 0.24434885395551187}

In [30]:
for i, j in ridge_results.items():
    print(i, j**(1/2))

0 0.4940759587836329
0.01 0.4940761127384496
0.1 0.49407749829044867
1 0.4940913503715112
10 0.4943165523786472
